In [0]:
import pandas as pd
import pickle as pk
import numpy as np
from joblib import dump, load


# Pivot

In [0]:
infile = open('/content/drive/My Drive/MScA/Machine Learning/ML Final Project/Data/Final Input Data/train_df_with_valid_recommendations.pkl','rb')
train_df = pk.load(infile)
infile.close()


infile = open('/content/drive/My Drive/MScA/Machine Learning/ML Final Project/Data/Final Input Data/validation_df_with_valid_recommendations.pkl','rb')
validation_df = pk.load(infile)
infile.close()


infile = open('/content/drive/My Drive/MScA/Machine Learning/ML Final Project/Data/Final Input Data/test_df_with_valid_recommendations.pkl','rb')
test_df = pk.load(infile)
infile.close()

## Sample calculation for test

In [0]:
df_small = train_df.sample(n=200,random_state=1)

In [0]:
temp_df = pd.DataFrame({ k : df_small.loc[df_small['productID'] == k, 'overall'] for k in df_small['productID'].unique() })
temp_df["reviewerID"] = df_small.reviewerID
temp_df.shape

(200, 197)

In [0]:
len(set(temp_df.reviewerID.tolist()))

198

In [0]:
pivots = df_small.pivot(index='reviewerID', columns='productID')['overall']

In [0]:
for i in range(len(pivots)):
  s = sum(pivots.iloc[i,:].isna())
  t = len(pivots.iloc[i,:].isna())
  if t - s > 1:
    print("found")
    print(i)
  else:
    continue

found
1
found
121


In [0]:
user = pivots.iloc[7,:]
user.dropna()

productID
B0050SWR6M    5.0
Name: A1585TVCKMAZOR, dtype: float64

## Deploy pivot

In [0]:
train_pivot = train_df.pivot_table(values='overall', index='reviewerID', columns='productID')

In [36]:
train_pivot

productID,0700026398,0700026657,0700099867,3828770193,8176503290,907843905X,B000003SQQ,B000006OVE,B000006OVG,B000006OWR,B000006OWT,B000006P0J,B000006P0M,B000006P0P,B000006RGO,B000006RGR,B000009QCX,B000009QD1,B00000DMA8,B00000DMAD,B00000DMAE,B00000DMAH,B00000DMAQ,B00000DMAR,B00000DMAV,B00000F1GS,B00000G16I,B00000I1B7,B00000I1BA,B00000I1BB,B00000I1BF,B00000I1BG,B00000I1BJ,B00000I1BK,B00000I1BN,B00000I1BQ,B00000I1BR,B00000I1BS,B00000I1BT,B00000I1BV,...,B01GW917NQ,B01GW91L50,B01GW91NK8,B01GW925WI,B01GW92ZKU,B01GWB19IW,B01GWB1GM6,B01GWB1UPO,B01GWCBSZU,B01GWGX74Q,B01GWGX7TG,B01GWGX836,B01GWGXBV0,B01GWGXH96,B01GWGXHKK,B01GWJCEOM,B01GWJCFB4,B01GWLIXY0,B01H0LFN3K,B01H2DKGZQ,B01H2DKHCS,B01H2DKHPA,B01H2DKHSM,B01H2DKHZ0,B01H2DKI3G,B01H2DKIDG,B01H2F5R6C,B01H3NU4OS,B01H440QZI,B01H5XD36C,B01HC8QZG0,B01HD1B64C,B01HD1B76O,B01HDJFJKG,B01HDJFJLK,B01HDJFJOM,B01HFRICLE,B01HH6JEOC,B01HIZF7XE,B01HIZGKOE
reviewerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A0059486XI1Z0P98KP35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A0220159ZRNBTRKLG08H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A0266076X6KPZ6CCHGVS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A0277912HT4JSJKVSL3E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A02836981FYG9912C66F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AZZHLZ8H23QD4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AZZKH219IDQGY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AZZT1ERHBSNQ8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

# Compute user score per feature

## Train

In [0]:
V = train_pivot.T
H = load("/content/drive/My Drive/MScA/Machine Learning/ML Final Project/Data/game_categories_probabilities.joblib")

In [38]:
V.head()

reviewerID,A0059486XI1Z0P98KP35,A0220159ZRNBTRKLG08H,A0266076X6KPZ6CCHGVS,A0277912HT4JSJKVSL3E,A02836981FYG9912C66F,A0331487QH9BVBTYGNCV,A03622674FSWUX0PXPAK,A0380485C177Q6QQNJIX,A0385843DE41TCVXH2I1,A0429448FT6MHSF724JB,A0435554Z2P98AIGLNCS,A049248150WLX2UGA57G,A0667676ELTQ9GS4VZHH,A0685888WB02Q69S553P,A0695568PX4DBZOQDN8,A0743345UFTOA4V1Z7W,A08761257GP04TJ0ROBS,A08981299ZTBZX8J5GSU,A0986665A7J4V94FK3RZ,A1002UBC6V258U,A1003D04KECOLP,A1006LIQ9Z6QN3,A100DXY4SLAMPM,A100JCBNALJFAW,A100LSS6IP0T7D,A100NE7N89DVC0,A100O5QLORBZRO,A100STMN7BADN9,A100TUZVB01NTC,A100U2O7L15XNL,A100Y9BI0I69B8,A1011Z5XSCK0WQ,A10175AMUHOQC4,A1017UZIPW58F4,A101D5E9YITTAZ,A101DN7FY59PUZ,A101EEOCU0H6E9,A101JMMLX8K4NY,A101R7OO4IRDTH,A101RMW2L4BES1,...,AZW10G02DNJI4,AZW1ZHG0U771R,AZW4K59K4QWZ8,AZWKRO5BBSBEB,AZWNYVPZYPTOZ,AZWTNCNFRKLZJ,AZWXG6KBXXC2N,AZX3PCPE29VL,AZX48KSN2OFA3,AZX7R9UN70VJD,AZX8Y5C30UHNA,AZXALX3NBP2O2,AZXBNJDQTYU8S,AZXG38H5X658K,AZXGVCWZ4QBR7,AZXHK8IO25FL6,AZY1N2YRSROMQ,AZY2B0MCDY4Z4,AZY7LNNZZE02P,AZYAFZSK1S805,AZYBK354R2Y95,AZYCXCDP9CSM1,AZYG5R26NUP70,AZYHFRGWH95QW,AZYIAWJR9972L,AZYKPCWUPTZEC,AZYOVGJLQ03ML,AZYSLPX9HUWPW,AZYU8M791SIFC,AZYUJNUA38IYL,AZYUNW5G2R3CL,AZYWRH9CQSB2Q,AZZ1KF8RAO1BR,AZZ2RPY63LR0B,AZZ3GFL1HTBLY,AZZHLZ8H23QD4,AZZKH219IDQGY,AZZT1ERHBSNQ8,AZZTC2OYVNE2Q,AZZTOUKVTUMVM
productID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0700026398,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0700026657,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0700099867,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3828770193,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8176503290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
H.set_index("productID",inplace = True)

In [40]:
H

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
productID,,,,,,,,,,,,,,,,,,,,,,
0439335310,0.000927,0.663799,0.000039,0.000029,0.000464,0.000555,0.001492,0.000912,0.000741,0.000051,0.000046,0.000274,0.000322,0.000042,0.001735,0.000717,0.000267,0.000449,0.322348,0.001474,0.002997,0.000320
0439339006,0.017509,0.078878,0.005404,0.019977,0.028995,0.016772,0.047489,0.029170,0.038341,0.004848,0.018106,0.018594,0.010457,0.041109,0.030652,0.024409,0.038886,0.042646,0.352370,0.029583,0.060733,0.045069
043933702X,0.012526,0.070359,0.011391,0.019072,0.018790,0.023021,0.027187,0.031538,0.045587,0.003711,0.015177,0.018577,0.013878,0.023528,0.018696,0.025422,0.027644,0.037165,0.386852,0.063006,0.071435,0.035437
0439339987,0.000302,0.000969,0.000311,0.000066,0.003829,0.000117,0.001396,0.000144,0.189689,0.000045,0.003567,0.000158,0.000266,0.000066,0.022651,0.000942,0.005781,0.000579,0.029984,0.057175,0.681795,0.000169
0439342260,0.007385,0.032591,0.003390,0.076898,0.010478,0.007003,0.012821,0.008481,0.023183,0.002279,0.004014,0.011004,0.012215,0.010681,0.205564,0.017459,0.012371,0.022678,0.353851,0.105753,0.049953,0.009948
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B01HI8JR82,0.002823,0.024435,0.000856,0.001784,0.020914,0.004905,0.021954,0.004953,0.022724,0.000760,0.001801,0.006555,0.003897,0.002276,0.007953,0.014628,0.002502,0.003093,0.663982,0.141306,0.040584,0.005314
B01HIZF83S,0.000028,0.000502,0.000024,0.000022,0.000010,0.000165,0.000827,0.000410,0.002180,0.000005,0.000018,0.000035,0.000057,0.000029,0.007135,0.000048,0.000947,0.000910,0.838712,0.001706,0.146192,0.000039
B01HIZGKOE,0.001858,0.003606,0.001327,0.001273,0.004730,0.001771,0.006998,0.000469,0.003753,0.000169,0.001397,0.003484,0.000894,0.006584,0.015598,0.046290,0.025824,0.038286,0.792713,0.011949,0.026726,0.004302


In [41]:
W = pd.DataFrame(columns = H.columns)
W

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21


In [0]:
users_no = V.columns

In [0]:
for i in range(len(users_no)):
    working_df = H.mul(V.iloc[:,i], axis=0)
    working_df.replace(0, np.NaN, inplace=True)    
    W.loc[users_no[i]] = working_df.mean(axis=0)

In [44]:
W

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
A0059486XI1Z0P98KP35,0.010114,0.185903,0.002355,1.560921e-02,1.395417e-01,0.011586,0.056420,0.013117,0.028177,1.110704e-03,7.751189e-03,0.020094,0.031537,2.336010e-02,0.067279,0.078562,0.165226,0.104860,3.289945,0.253884,0.166338,0.327231
A0220159ZRNBTRKLG08H,0.000039,0.000014,0.000048,4.864822e-06,8.081573e-07,0.000011,0.000036,0.000010,0.002789,1.759331e-06,1.332568e-05,0.000003,0.000006,1.427458e-05,0.006426,0.000050,0.000009,0.000121,3.838918,0.004078,0.147207,0.000199
A0266076X6KPZ6CCHGVS,0.000057,0.000033,0.000008,8.322102e-08,4.436573e-07,0.000076,0.000013,0.746313,0.000038,8.691167e-07,5.523598e-07,0.000009,0.000015,7.375747e-07,0.000520,0.000044,0.000035,0.000018,3.517132,0.000233,0.735397,0.000056
A0277912HT4JSJKVSL3E,0.016276,0.282223,0.006960,4.225503e-01,3.025044e-01,0.012527,0.194502,0.025375,0.484571,3.254560e-03,9.193853e-03,0.220467,0.012149,1.159728e-01,0.040106,0.033908,0.343935,0.044280,1.738825,0.026848,0.153594,0.009977
A02836981FYG9912C66F,0.006836,0.024869,0.001667,7.308678e-02,2.517473e-02,1.683360,0.448498,0.023459,0.074326,8.633794e-04,1.829895e-03,0.012193,0.020936,7.006482e-03,0.030648,0.030767,0.007151,0.034565,1.899184,0.029240,0.062545,0.001794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AZZHLZ8H23QD4,0.215093,0.411835,0.005991,1.002124e-02,2.172979e-02,0.008097,0.033823,0.016060,0.099039,1.261692e-03,4.647397e-03,0.021262,0.006142,8.326782e-03,0.054419,0.019158,0.006272,0.020564,3.260633,0.060400,0.467872,0.025133
AZZKH219IDQGY,0.003421,0.041225,0.000654,3.913172e-03,1.829780e-02,0.013648,0.013572,0.007552,0.033698,7.010607e-04,3.554569e-03,0.012890,0.015055,9.594536e-03,0.034779,0.036530,0.002164,0.018939,3.666405,0.022780,0.035325,0.005301
AZZT1ERHBSNQ8,0.013890,0.097940,0.009016,1.086251e-02,5.789041e-02,0.009779,0.076231,0.041793,1.474746,2.098405e-03,1.873722e-02,0.512632,0.010666,1.563086e-02,0.055826,0.046694,0.031222,0.065053,1.038262,0.088169,1.305927,0.016935
AZZTC2OYVNE2Q,0.000011,0.000181,0.000098,1.221302e-05,3.700682e-06,0.004099,0.000113,0.000055,0.000178,4.760863e-06,1.096018e-05,0.000009,0.000240,1.769816e-06,0.004267,0.000670,0.000128,0.000373,3.905380,0.002937,0.079605,0.001621


In [45]:
dump(W, "/content/drive/My Drive/MScA/Machine Learning/ML Final Project/Data/user_cat_scores.joblib")

['/content/drive/My Drive/MScA/Machine Learning/ML Final Project/Data/user_cat_scores.joblib']

In [0]:
W = load("/content/drive/My Drive/MScA/Machine Learning/ML Final Project/Data/user_cat_scores.joblib")

# Predict

## Product and ratings-based

In [47]:
top_recs_df = pd.DataFrame(columns=["reviewerID","top_1_recommended","top_3_recommended",
                                    "top_5_recommended","top_10_recommended",
                                    "top_15_recommended"])
for i in range(len(users_no)):
    working_df = H.mul(W.iloc[i], axis=1)
    user_recs = working_df.sum(axis=1)
    recs = np.array(user_recs)
    prod_idx = np.array(user_recs.index)
    top_index = recs.argsort()[-15:][::-1]
    top_recs_1 = [prod_idx[i] for i in [top_index[0]]]
    top_recs_3 = [prod_idx[i] for i in top_index[:3]]
    top_recs_5 = [prod_idx[i] for i in top_index[:6]]
    top_recs_10 = [prod_idx[i] for i in top_index[:11]]
    top_recs_15 = [prod_idx[i] for i in top_index]
    top_user = pd.DataFrame({"reviewerID":users_no[i],
                             "top_1_recommended":[top_recs_1],
                             "top_3_recommended":[top_recs_3],
                             "top_5_recommended":[top_recs_5],
                             "top_10_recommended":[top_recs_10],
                             "top_15_recommended":[top_recs_15]})
    top_recs_df = top_recs_df.append(top_user, ignore_index=True)    
top_recs_df

,reviewerID,top_1_recommended,top_3_recommended,top_5_recommended,top_10_recommended,top_15_recommended
0,A0059486XI1Z0P98KP35,[B002HJQSQW],"[B002HJQSQW, B003ITXT26, B0025KZV84]","[B002HJQSQW, B003ITXT26, B0025KZV84, B0030GBU2...","[B002HJQSQW, B003ITXT26, B0025KZV84, B0030GBU2...","[B002HJQSQW, B003ITXT26, B0025KZV84, B0030GBU2..."
1,A0220159ZRNBTRKLG08H,[B002HJQSQW],"[B002HJQSQW, B003ITXT26, B0025KZV84]","[B002HJQSQW, B003ITXT26, B0025KZV84, B0030GBU2...","[B002HJQSQW, B003ITXT26, B0025KZV84, B0030GBU2...","[B002HJQSQW, B003ITXT26, B0025KZV84, B0030GBU2..."
2,A0266076X6KPZ6CCHGVS,[B002HJQSQW],"[B002HJQSQW, B003ITXT26, B0030GBU2I]","[B002HJQSQW, B003ITXT26, B0030GBU2I, B0025KZV8...","[B002HJQSQW, B003ITXT26, B0030GBU2I, B0025KZV8...","[B002HJQSQW, B003ITXT26, B0030GBU2I, B0025KZV8..."
3,A0277912HT4JSJKVSL3E,[B002HJQSQW],"[B002HJQSQW, B003ITXT26, B0025KZV84]","[B002HJQSQW, B003ITXT26, B0025KZV84, B0030GBU2...","[B002HJQSQW, B003ITXT26, B0025KZV84, B0030GBU2...","[B002HJQSQW, B003ITXT26, B0025KZV84, B0030GBU2..."
4,A02836981FYG9912C66F,[B002HJQSQW],"[B002HJQSQW, B003ITXT26, B0025KZV84]","[B002HJQSQW, B003ITXT26, B0025KZV84, B003O6EFK...","[B002HJQSQW, B003ITXT26, B0025KZV84, B003O6EFK...","[B002HJQSQW, B003ITXT26, B0025KZV84, B003O6EFK..."
...,...,...,...,...,...,...
47694,AZZHLZ8H23QD4,[B002HJQSQW],"[B002HJQSQW, B003ITXT26, B0030GBU2I]","[B002HJQSQW, B003ITXT26, B0030GBU2I, B0025KZV8...","[B002HJQSQW, B003ITXT26, B0030GBU2I, B0025KZV8...","[B002HJQSQW, B003ITXT26, B0030GBU2I, B0025KZV8..."
47695,AZZKH219IDQGY,[B002HJQSQW],"[B002HJQSQW, B003ITXT26, B0025KZV84]","[B002HJQSQW, B003ITXT26, B0025KZV84, B0030GBU2...","[B002HJQSQW, B003ITXT26, B0025KZV84, B0030GBU2...","[B002HJQSQW, B003ITXT26, B0025KZV84, B0030GBU2..."
47696,AZZT1ERHBSNQ8,[B000ZKPBWM],"[B000ZKPBWM, B00KWF366S, B01307QVCA]","[B000ZKPBWM, B00KWF366S, B01307QVCA, B00LIWF1G...","[B000ZKPBWM, B00KWF366S, B01307QVCA, B00LIWF1G...","[B000ZKPBWM, B00KWF366S, B01307QVCA, B00LIWF1G..."
47697,AZZTC2OYVNE2Q,[B002HJQSQW],"[B002HJQSQW, B003ITXT26, B0025KZV84]","[B002HJQSQW, B003ITXT26, B0025KZV84, B0030GBU2...","[B002HJQSQW, B003ITXT26, B0025KZV84, B0030GBU2...","[B002HJQSQW, B003ITXT26, B0025KZV84, B0030GBU2..."


In [51]:
dump(top_recs_df, "/content/drive/My Drive/MScA/Machine Learning/ML Final Project/Data/cb_top_user_preds.joblib")

['/content/drive/My Drive/MScA/Machine Learning/ML Final Project/Data/cb_top_user_preds.joblib']

In [49]:
"A1ZP3F3PSDURB4" in users_no

True

In [50]:
"AZZTC2OYVNE2Q" in users_no

True

## Product-only (Cosine Similarity)

In [0]:
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(H, H)

In [0]:
import numpy as np
product_list = np.array(H.index).tolist()

In [0]:
len(product_list)

27238

In [0]:
cosine_sim.shape

(27238, 27238)

In [0]:
product_list.index('0439335310')

0

In [0]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = product_list.index(title)

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    game_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    rec = []
    for i in game_indices:
      rec.append(product_list[i])
    return rec

In [0]:
top_recs_prod_df = pd.DataFrame()
for i in range(len(users_no)):
  rslt = get_recommendations(users_no[i])
  top_recs_prod_df[users_no[i]] = rslt

top_recs_prod_df

ValueError: ignored